In [0]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 185kB/s 
ERROR: tensorflow-model-optimization 0.2.1 requires enum34~=1.1, which is not installed.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [0]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

TensorFlow 2.x selected.
2.1.0
/device:GPU:0
Num GPUs Available:  1


In [0]:
from tensorflow.keras.callbacks import Callback

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
np.random.seed(12227)

In [0]:
n_ep = 200
loss = 0.2
bs = 1000

In [0]:
def custom_stopping(value=0.5, verbose=0):
    early = EarlyStoppingByLossVal(monitor='val_loss', value=value, verbose=verbose)
    return early

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_acc', value=0.95, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        # if current is None:
        # warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [0]:
def DataPreparation(data_input_file):
  print('Bevilacqua  et al. 2015 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  # X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  # classes_number = Y.shape[1]
  # Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [0]:
X,Y,f=DataPreparation('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Bevilacqua  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz


In [0]:
print(X[2].shape)

(1, 250, 23)


In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def RunBevilacqua(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
def build_model_1(row,col,num_classes):

 
 
  layers = [
        tf.keras.layers.Conv2D(filters=10,kernel_size=(3, 5), activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(3,3),strides = 1),

        tf.keras.layers.Conv2D(filters=2,kernel_size=(2, 4),activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = 1),

        tf.keras.layers.Conv2D(filters=2,kernel_size=(2, 2),activation = tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(3,2),strides = 1),
        
        

       

       
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(500, activation = tf.nn.relu),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(25, activation = tf.nn.relu),
        tf.keras.layers.Dropout(0.5),
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [0]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _, _,img_rows, img_cols = X.shape
  print(X.shape)
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=build_model_1(img_rows,img_cols,n_class)
    # print(model.summary)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')
    model.fit(X_train, y[train_idx], batch_size=64, epochs=300 )
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [0]:
tf.keras.backend.set_image_data_format('channels_first')

In [0]:
RunBevilacqua('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Streaming output truncated to the last 5000 lines.
Epoch 216/300
2292/2292 [==============================] - 0s 146us/sample - loss: 0.3011 - accuracy: 0.8626
Epoch 217/300
2292/2292 [==============================] - 0s 144us/sample - loss: 0.3199 - accuracy: 0.8691
Epoch 218/300
2292/2292 [==============================] - 0s 156us/sample - loss: 0.3016 - accuracy: 0.8674
Epoch 219/300
2292/2292 [==============================] - 0s 147us/sample - loss: 0.2659 - accuracy: 0.8791
Epoch 220/300
2292/2292 [==============================] - 0s 146us/sample - loss: 0.3045 - accuracy: 0.8682
Epoch 221/300
2292/2292 [==============================] - 0s 149us/sample - loss: 0.2523 - accuracy: 0.8822
Epoch 222/300
2292/2292 [==============================] - 0s 147us/sample - loss: 0.2616 - accuracy: 0.8787
Epoch 223/300
2292/2292 [==============================] - 0s 146us/sample - loss: 0.3059 - accuracy: 0.8661
Epoch 224/300
2292/2292 [==============================] - 0s 147us/sample - 

In [0]:
RunBevilacqua('/content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz')

Streaming output truncated to the last 5000 lines.
Epoch 216/300
1200/1200 [==============================] - 0s 144us/sample - loss: 0.1951 - accuracy: 0.9383
Epoch 217/300
1200/1200 [==============================] - 0s 147us/sample - loss: 0.1670 - accuracy: 0.9442
Epoch 218/300
1200/1200 [==============================] - 0s 143us/sample - loss: 0.1567 - accuracy: 0.9492
Epoch 219/300
1200/1200 [==============================] - 0s 152us/sample - loss: 0.1699 - accuracy: 0.9458
Epoch 220/300
1200/1200 [==============================] - 0s 143us/sample - loss: 0.1632 - accuracy: 0.9467
Epoch 221/300
1200/1200 [==============================] - 0s 148us/sample - loss: 0.1731 - accuracy: 0.9392
Epoch 222/300
1200/1200 [==============================] - 0s 140us/sample - loss: 0.1599 - accuracy: 0.9500
Epoch 223/300
1200/1200 [==============================] - 0s 141us/sample - loss: 0.1554 - accuracy: 0.9492
Epoch 224/300
1200/1200 [==============================] - 0s 143us/sample - 

In [0]:
RunBevilacqua('/content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz')

Streaming output truncated to the last 5000 lines.
Epoch 216/300
2298/2298 [==============================] - 0s 145us/sample - loss: 0.1452 - accuracy: 0.9513
Epoch 217/300
2298/2298 [==============================] - 0s 144us/sample - loss: 0.1403 - accuracy: 0.9517
Epoch 218/300
2298/2298 [==============================] - 0s 141us/sample - loss: 0.1186 - accuracy: 0.9595
Epoch 219/300
2298/2298 [==============================] - 0s 152us/sample - loss: 0.1393 - accuracy: 0.9517
Epoch 220/300
2298/2298 [==============================] - 0s 143us/sample - loss: 0.1301 - accuracy: 0.9574
Epoch 221/300
2298/2298 [==============================] - 0s 141us/sample - loss: 0.1193 - accuracy: 0.9617
Epoch 222/300
2298/2298 [==============================] - 0s 140us/sample - loss: 0.1237 - accuracy: 0.9565
Epoch 223/300
2298/2298 [==============================] - 0s 150us/sample - loss: 0.1295 - accuracy: 0.9604
Epoch 224/300
2298/2298 [==============================] - 0s 145us/sample - 

In [0]:
RunBevilacqua('/content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz')

Streaming output truncated to the last 5000 lines.
Epoch 216/300
2299/2299 [==============================] - 0s 141us/sample - loss: 0.2825 - accuracy: 0.8617
Epoch 217/300
2299/2299 [==============================] - 0s 145us/sample - loss: 0.2936 - accuracy: 0.8578
Epoch 218/300
2299/2299 [==============================] - 0s 151us/sample - loss: 0.2815 - accuracy: 0.8569
Epoch 219/300
2299/2299 [==============================] - 0s 145us/sample - loss: 0.2964 - accuracy: 0.8569
Epoch 220/300
2299/2299 [==============================] - 0s 146us/sample - loss: 0.2768 - accuracy: 0.8595
Epoch 221/300
2299/2299 [==============================] - 0s 155us/sample - loss: 0.2750 - accuracy: 0.8578
Epoch 222/300
2299/2299 [==============================] - 0s 145us/sample - loss: 0.2842 - accuracy: 0.8543
Epoch 223/300
2299/2299 [==============================] - 0s 146us/sample - loss: 0.2778 - accuracy: 0.8621
Epoch 224/300
2299/2299 [==============================] - 0s 148us/sample - 